Step 1. Input

In [ ]:
# import dependencies 
import torch
import torchvision
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# OPTIONAL: show annotations on image
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

In [ ]:
# sample input
image = cv2.imread('../images/dog.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

# shows the image to be detected

Step 2. Segment Anything Output

In [ ]:
# provide SAM model to the mask generator class
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "../../sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# run automatic mask generation
masks = mask_generator.generate(image)

In [ ]:
length_masks = len(masks)
for mask in length_masks: 
  # crop the image AKA get the bounding box
  bbox_mask = masks[mask]['bbox']
  # ask OpenClip "This is a photo of what?"

Step 3. Crop images for each mask

In [1]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

ModuleNotFoundError: No module named 'open_clip'

Step 4. Use CLIP to label the mask : clip_label

Step 5. Compare clip_label to COCO

In [ ]:
# If clip_label in COCO: Compare the prediction with ground truth Score using mAP -> get ground score
# Else: Ignore